In [1]:
# Install required packages
!pip install ultralytics roboflow opencv-python-headless matplotlib seaborn pandas numpy Pillow
!pip install plotly kaleido scikit-learn ipywidgets

# Import essential libraries
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print(" All dependencies installed successfully!")
print(" Libraries imported and ready to use!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

 All dependencies installed successfully!
 Libraries imported and ready to use!


In [17]:
# Import YOLOv8 and other ML libraries
from ultralytics import YOLO
from roboflow import Roboflow
import torch
from sklearn.metrics import classification_report, confusion_matrix
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.colab import files, drive
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"  Using device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  GPU not available. Training will be slower on CPU.")

print("\n YOLOv8 and ML libraries ready!")

  Using device: cuda
   GPU: Tesla T4
   Memory: 15.8 GB

 YOLOv8 and ML libraries ready!


In [3]:
# Create project directories
project_dir = Path('/content/asl_project')
project_dir.mkdir(exist_ok=True)

# Directory structure
dirs = {
    'datasets': project_dir / 'datasets',
    'models': project_dir / 'models',
    'results': project_dir / 'results',
    'visualizations': project_dir / 'visualizations',
    'predictions': project_dir / 'predictions'
}

for dir_path in dirs.values():
    dir_path.mkdir(exist_ok=True)

print(" Project structure created:")
for name, path in dirs.items():
    print(f"   {name}: {path}")

os.chdir(project_dir)
print(f"\n Working directory: {os.getcwd()}")

 Project structure created:
   datasets: /content/asl_project/datasets
   models: /content/asl_project/models
   results: /content/asl_project/results
   visualizations: /content/asl_project/visualizations
   predictions: /content/asl_project/predictions

 Working directory: /content/asl_project


In [4]:
# Download comprehensive ASL datasets from Roboflow
print(" Downloading ASL datasets...")

# Initialize Roboflow Access
rf = Roboflow("k2qq61C8Fe7H3IDfG3Gj")

# Download ASL Alphabet dataset
print("\n Downloading ASL Alphabet dataset...")
try:
    alphabet_project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")
    alphabet_dataset = alphabet_project.version(6).download("yolov8", location="./datasets/asl_alphabet")
    print(" ASL Alphabet dataset downloaded successfully!")
except Exception as e:
    print(f"  Alphabet dataset error: {e}")
    print("   Trying alternative source...")

# Downloading ASL Numbers dataset
print("\n Downloading ASL Numbers dataset...")
try:
    numbers_project = rf.workspace("roboflow-58fyf").project("asl-numbers")
    numbers_dataset = numbers_project.version(1).download("yolov8", location="./datasets/asl_numbers")
    print(" ASL Numbers dataset downloaded successfully!")
except Exception as e:
    print(f"  Numbers dataset error: {e}")
    print("   Trying alternative source...")

# Backup Plan: Download from alternative sources if Roboflow fails
print("\n Checking for backup datasets...")
if not (project_dir / 'datasets' / 'asl_alphabet').exists():
    print(" Downloading backup ASL alphabet dataset...")
    !wget -O asl_alphabet.zip https://github.com/grassknoted/ASL-Alphabet/archive/master.zip
    !unzip -q asl_alphabet.zip -d ./datasets/
    print(" Backup alphabet dataset ready!")

print("\n Dataset download completed!")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/asl_alphabet in yolov8:: 100%|██████████| 1452/1452 [00:00<00:00, 1962.90it/s]

 ASL Alphabet dataset downloaded successfully!

loading Roboflow workspace...


loading Roboflow project...
  Numbers dataset error: Version number 1 is not found.
   Trying alternative source...

 Checking for backup datasets...

 Dataset download completed!


In [18]:
# Create unified dataset structure for YOLOv8
import shutil
import yaml
from collections import defaultdict, Counter

print(" Preparing unified ASL dataset...")

# Define all ASL classes (36 total)
asl_classes = {
    # Alphabet A-Z
    'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8,
    'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16,
    'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25,
    # Numbers 0-9
    '0': 26, '1': 27, '2': 28, '3': 29, '4': 30, '5': 31, '6': 32, '7': 33, '8': 34, '9': 35
}

# Create unified dataset directory
unified_dir = project_dir / 'datasets' / 'asl_unified'
unified_dir.mkdir(exist_ok=True)

for split in ['train', 'val', 'test']:
    (unified_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (unified_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Function to convert and copy dataset
def process_dataset(source_dir, dataset_name, class_mapping=None):
    """Process and integrate dataset into unified structure"""
    print(f"\n   Processing {dataset_name}...")

    if not source_dir.exists():
        return 0

    files_processed = 0

    for split in ['train', 'valid', 'test']:
        split_name = 'val' if split == 'valid' else split

        source_images = source_dir / split / 'images'
        source_labels = source_dir / split / 'labels'

        if source_images.exists():
            # Copy images
            for img_file in source_images.glob('*'):
                if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    dest_img = unified_dir / split_name / 'images' / f"{dataset_name}_{img_file.name}"
                    shutil.copy2(img_file, dest_img)

                    # Copy corresponding label
                    label_file = source_labels / f"{img_file.stem}.txt"
                    if label_file.exists():
                        dest_label = unified_dir / split_name / 'labels' / f"{dataset_name}_{img_file.stem}.txt"

                        # Update class IDs if mapping provided
                        if class_mapping:
                            with open(label_file, 'r') as f:
                                lines = f.readlines()

                            with open(dest_label, 'w') as f:
                                for line in lines:
                                    parts = line.strip().split()
                                    if parts:
                                        old_class = int(parts[0])
                                        if old_class in class_mapping:
                                            parts[0] = str(class_mapping[old_class])
                                            f.write(' '.join(parts) + '\n')
                        else:
                            shutil.copy2(label_file, dest_label)

                    files_processed += 1

    return files_processed

# Process available datasets
total_files = 0

# Process alphabet dataset
alphabet_dir = project_dir / 'datasets' / 'asl_alphabet'
total_files += process_dataset(alphabet_dir, 'alphabet')

# Process numbers dataset
numbers_dir = project_dir / 'datasets' / 'asl_numbers'
total_files += process_dataset(numbers_dir, 'numbers')

print(f"\n Unified dataset created with {total_files} files!")

# Create YAML configuration for YOLOv8
yaml_config = {
    'path': str(unified_dir),
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': 36,  # number of classes
    'names': [k for k, v in sorted(asl_classes.items(), key=lambda x: x[1])]
}

yaml_path = unified_dir / 'asl_config.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_config, f, default_flow_style=False)

print(f" YOLOv8 config saved: {yaml_path}")
print(f" Classes configured: {len(asl_classes)} total")
print(f"   Alphabet: A-Z (26 classes)")
print(f"   Numbers: 0-9 (10 classes)")

 Preparing unified ASL dataset...

   Processing alphabet...

   Processing numbers...
     numbers not found, skipping...

 Unified dataset created with 720 files!
 YOLOv8 config saved: /content/asl_project/datasets/asl_unified/asl_config.yaml
 Classes configured: 36 total
   Alphabet: A-Z (26 classes)
   Numbers: 0-9 (10 classes)


In [6]:
# Analyze the unified dataset
def analyze_dataset(dataset_path):
    """Analyze dataset composition and create visualizations"""

    print(" Analyzing ASL dataset composition...")

    splits_data = {}

    for split in ['train', 'val', 'test']:
        split_dir = dataset_path / split

        if split_dir.exists():
            images = list((split_dir / 'images').glob('*'))
            labels = list((split_dir / 'labels').glob('*'))

            # Count classes in this split
            class_counts = defaultdict(int)
            total_annotations = 0

            for label_file in labels:
                with open(label_file, 'r') as f:
                    for line in f:
                        class_id = int(line.strip().split()[0])
                        class_counts[class_id] += 1
                        total_annotations += 1

            splits_data[split] = {
                'images': len(images),
                'labels': len(labels),
                'annotations': total_annotations,
                'class_counts': dict(class_counts)
            }

    return splits_data

# Analyze dataset
analysis = analyze_dataset(unified_dir)

# Create comprehensive visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Dataset Split Distribution',
        'Images vs Labels',
        'Class Distribution (Training Set)',
        'Total Annotations by Split'
    ],
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 1. Dataset split distribution (pie chart)
splits = list(analysis.keys())
split_counts = [analysis[split]['images'] for split in splits]

fig.add_trace(
    go.Pie(labels=splits, values=split_counts, name="Split Distribution"),
    row=1, col=1
)

# 2. Images vs Labels comparison
images_count = [analysis[split]['images'] for split in splits]
labels_count = [analysis[split]['labels'] for split in splits]

fig.add_trace(
    go.Bar(x=splits, y=images_count, name='Images', marker_color='lightblue'),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=splits, y=labels_count, name='Labels', marker_color='lightcoral'),
    row=1, col=2
)

# 3. Class distribution in training set
if 'train' in analysis and analysis['train']['class_counts']:
    class_names = [yaml_config['names'][i] for i in sorted(analysis['train']['class_counts'].keys())]
    class_counts = [analysis['train']['class_counts'][i] for i in sorted(analysis['train']['class_counts'].keys())]

    fig.add_trace(
        go.Bar(x=class_names, y=class_counts, name='Training Classes', marker_color='lightgreen'),
        row=2, col=1
    )

# 4. Total annotations by split
annotation_counts = [analysis[split]['annotations'] for split in splits]

fig.add_trace(
    go.Bar(x=splits, y=annotation_counts, name='Annotations', marker_color='gold'),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text="ASL Dataset Analysis Dashboard",
    showlegend=True,
    height=800
)

fig.show()

# Print summary statistics
print("\n Dataset Summary:")
total_images = sum(analysis[split]['images'] for split in analysis)
total_annotations = sum(analysis[split]['annotations'] for split in analysis)

print(f"   Total Images: {total_images:,}")
print(f"   Total Annotations: {total_annotations:,}")
print(f"   Classes: {len(asl_classes)} (A-Z + 0-9)")

for split in analysis:
    data = analysis[split]
    print(f"\n   {split.upper()} Set:")
    print(f"     Images: {data['images']:,}")
    print(f"     Labels: {data['labels']:,}")
    print(f"     Annotations: {data['annotations']:,}")

    if data['annotations'] > 0:
        avg_per_image = data['annotations'] / data['images'] if data['images'] > 0 else 0
        print(f"     Avg annotations/image: {avg_per_image:.2f}")

 Analyzing ASL dataset composition...



 Dataset Summary:
   Total Images: 720
   Total Annotations: 720
   Classes: 36 (A-Z + 0-9)

   TRAIN Set:
     Images: 504
     Labels: 504
     Annotations: 504
     Avg annotations/image: 1.00

   VAL Set:
     Images: 144
     Labels: 144
     Annotations: 144
     Avg annotations/image: 1.00

   TEST Set:
     Images: 72
     Labels: 72
     Annotations: 72
     Avg annotations/image: 1.00


In [7]:
# Initialize YOLOv8 model
print(" Initializing YOLOv8 model...")

# Use YOLOv8n (nano) for faster training, or YOLOv8s/m/l for better accuracy
model_size = 'n'  # Options: n (nano), s (small), m (medium), l (large), x (extra large)
model = YOLO(f'yolov8{model_size}.pt')  # Load pretrained model

print(f" YOLOv8{model_size} model loaded successfully!")
print(f" Target classes: {len(asl_classes)} ASL signs")

# Training hyperparameters optimized for ASL detection
training_config = {
    'data': str(yaml_path),
    'epochs': 100,  # Increased for better accuracy
    'imgsz': 640,   # Image size
    'batch': 16,    # Adjust based on GPU memory
    'lr0': 0.01,    # Initial learning rate
    'lrf': 0.1,     # Final learning rate factor
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'box': 0.05,    # Box loss gain
    'cls': 0.5,     # Class loss gain
    'dfl': 1.5,     # DFL loss gain
    'pose': 12.0,   # Pose loss gain (unused)
    'kobj': 1.0,    # Keypoint obj loss gain (unused)
    'label_smoothing': 0.0,
    'nbs': 64,      # Nominal batch size
    'overlap_mask': True,
    'mask_ratio': 4,
    'dropout': 0.0,
    'val': True,    # Validate during training
    'save': True,   # Save checkpoints
    'save_period': 10,  # Save every 10 epochs
    'cache': False, # Cache images for faster training (disable if memory limited)
    'device': device,
    'workers': 8,   # Number of worker threads
    'project': str(dirs['models']),
    'name': 'asl_yolov8_training',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'auto',  # Auto-select optimizer
    'verbose': True,
    'seed': 42,     # For reproducibility
    'deterministic': True,
    'single_cls': False,
    'rect': False,  # Rectangular training
    'cos_lr': False,
    'close_mosaic': 10,
    'resume': False,
    'amp': True,    # Automatic Mixed Precision
    'fraction': 1.0,
    'profile': False,
    'freeze': None,
}

print("\n  Training Configuration:")
for key, value in training_config.items():
    print(f"   {key}: {value}")

print("\n Ready to start training!")
print(f" Models will be saved to: {dirs['models']}")
print(f" Training results will be in: {dirs['results']}")

 Initializing YOLOv8 model...


 YOLOv8n model loaded successfully!
 Target classes: 36 ASL signs

  Training Configuration:
   data: /content/asl_project/datasets/asl_unified/asl_config.yaml
   epochs: 100
   imgsz: 640
   batch: 16
   lr0: 0.01
   lrf: 0.1
   momentum: 0.937
   weight_decay: 0.0005
   warmup_epochs: 3.0
   warmup_momentum: 0.8
   box: 0.05
   cls: 0.5
   dfl: 1.5
   pose: 12.0
   kobj: 1.0
   label_smoothing: 0.0
   nbs: 64
   overlap_mask: True
   mask_ratio: 4
   dropout: 0.0
   val: True
   save: True
   save_period: 10
   cache: False
   device: cuda
   workers: 8
   project: /content/asl_project/models
   name: asl_yolov8_training
   exist_ok: True
   pretrained: True
   optimizer: auto
   verbose: True
   seed: 42
   deterministic: True
   single_cls: False
   rect: False
   cos_lr: False
   close_mosaic: 10
   resume: False
   amp: True
   fraction: 1.0
   profile: False
   freeze: None

 Ready to start training!
 Models will be saved to: /content/asl_project/models
 Training results will be

In [8]:
# Start training with progress monitoring
print(" Starting YOLOv8 training for ASL detection...")
print("  This will take 30-60 minutes depending on your GPU")
print(" Training metrics will be displayed in real-time\n")

try:
    # Train the model
    results = model.train(**training_config)

    print("\n Training completed successfully!")
    print(f" Best model saved as: best.pt")
    print(f" Training results saved to: {training_config['project']}/{training_config['name']}")

except Exception as e:
    print(f" Training failed: {e}")
    print(" Trying with reduced batch size...")

    # Retry with smaller batch size if memory issues
    training_config['batch'] = 8
    training_config['imgsz'] = 512

    try:
        results = model.train(**training_config)
        print("\n Training completed with reduced settings!")
    except Exception as e2:
        print(f" Training failed again: {e2}")
        print("  Please check your dataset and try again")

 Starting YOLOv8 training for ASL detection...
  This will take 30-60 minutes depending on your GPU
 Training metrics will be displayed in real-time

WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.180 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.05, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/asl_project/datasets/asl_unified/asl_config.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, ma

Overriding model.yaml nc=80 with nc=36

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    758332  ultralytics.nn.modules.head.Detect           [36, [64, 128, 256]]          
Model summary: 129 layers, 3,017,868 parameters, 3,017,852 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2554.9±1556.4 MB/s, size: 197.2 KB)


train: Scanning /content/asl_project/datasets/asl_unified/train/labels... 504 images, 0 backgrounds, 0 corrupt: 100%|██████████| 504/504 [00:00<00:00, 2367.08it/s]

...

    100/100      3.97G   0.001747     0.6057     0.8452          8        640: 100%|██████████| 32/32 [00:12<00:00,  2.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.25it/s]

                   all        144        144      0.927      0.894      0.964      0.917



100 epochs completed in 0.425 hours.
Optimizer stripped from /content/asl_project/models/asl_yolov8_training/weights/last.pt, 6.3MB
Optimizer stripped from /content/asl_project/models/asl_yolov8_training/weights/best.pt, 6.3MB

Validating /content/asl_project/models/asl_yolov8_training/weights/best.pt...
Ultralytics 8.3.180 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,012,668 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


                   all        144        144      0.927      0.894      0.964      0.917
                     A          5          5          1      0.679      0.995      0.932
                     B          9          9          1      0.914      0.995       0.93
                     C          3          3      0.942          1      0.995      0.913
                     D          6          6      0.836          1      0.995      0.956
                     E          4          4      0.974          1      0.995      0.995
                     F          8          8      0.966          1      0.995      0.976
                     G          5          5      0.961          1      0.995      0.941
                     H          9          9      0.982          1      0.995      0.924
                     I          2          2      0.881        0.5      0.524      0.524
                     J          8          8      0.983          1      0.995      0.698
                     

In [9]:
# Load the best trained model
print(" Loading best trained model...")

try:
    # Find the best model path
    model_path = dirs['models'] / 'asl_yolov8_training' / 'weights' / 'best.pt'

    if model_path.exists():
        best_model = YOLO(str(model_path))
        print(f" Best model loaded from: {model_path}")
    else:
        print(" Best model not found, using last checkpoint...")
        model_path = dirs['models'] / 'asl_yolov8_training' / 'weights' / 'last.pt'
        best_model = YOLO(str(model_path))

except Exception as e:
    print(f" Error loading model: {e}")
    print("Using original model for evaluation...")
    best_model = model

# Model information
print("\n Model Information:")
print(f"   Architecture: YOLOv8{model_size}")
print(f"   Classes: {len(asl_classes)}")
print(f"   Device: {device}")

# Quick model summary
try:
    model_info = best_model.info()
    if model_info:
        print(f"   Parameters: {model_info.get('params', 'Unknown'):,}")
        print(f"   FLOPs: {model_info.get('flops', 'Unknown')}")
except:
    print("   Model summary not available")

print("\n Model ready for evaluation!")

 Loading best trained model...
 Best model loaded from: /content/asl_project/models/asl_yolov8_training/weights/best.pt

 Model Information:
   Architecture: YOLOv8n
   Classes: 36
   Device: cuda
Model summary: 129 layers, 3,017,868 parameters, 0 gradients, 8.2 GFLOPs
   Model summary not available

 Model ready for evaluation!


In [10]:
# Comprehensive model evaluation
print(" Running comprehensive model evaluation...")

# Validate on test set
try:
    print("\n Validating on test set...")
    validation_results = best_model.val(
        data=str(yaml_path),
        split='test',
        imgsz=640,
        batch=16,
        conf=0.25,
        iou=0.6,
        device=device,
        verbose=True,
        save_json=True,
        save_hybrid=False,
        project=str(dirs['results']),
        name='validation'
    )

    print(" Validation completed!")

    # Extract key metrics
    if hasattr(validation_results, 'box'):
        metrics = validation_results.box
        map50_95 = metrics.map  # mAP@0.5-0.95
        map50 = metrics.map50   # mAP@0.5
        precision = metrics.mp  # Mean precision
        recall = metrics.mr     # Mean recall

        print(f"\n Performance Metrics:")
        print(f"   mAP@0.5-0.95: {map50_95:.3f} ({map50_95*100:.1f}%)")
        print(f"   mAP@0.5: {map50:.3f} ({map50*100:.1f}%)")
        print(f"   Precision: {precision:.3f} ({precision*100:.1f}%)")
        print(f"   Recall: {recall:.3f} ({recall*100:.1f}%)")

        # Checking if we met the 90% accuracy target
        if map50 >= 0.90:
            print("\n SUCCESS: Model achieved 90%+ accuracy target!")
        elif map50 >= 0.85:
            print("\n GOOD: Model achieved 85%+ accuracy (close to target)")
        elif map50 >= 0.75:
            print("\n  OKAY: Model achieved 75%+ accuracy (needs improvement)")
        else:
            print("\n  LOW: Model accuracy below 75% (requires more training/data)")

except Exception as e:
    print(f" Validation error: {e}")
    print("Continuing with other evaluations...")

 Running comprehensive model evaluation...

 Validating on test set...
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.180 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,012,668 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2876.8±1010.0 MB/s, size: 231.9 KB)


val: Scanning /content/asl_project/datasets/asl_unified/test/labels... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<00:00, 2368.44it/s]

val: New cache created: /content/asl_project/datasets/asl_unified/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


                   all         72         72      0.942      0.889      0.942      0.915
                     A          1          1          1          1      0.995      0.995
                     B          3          3          1          1      0.995      0.995
                     C          4          4      0.933          1      0.995      0.972
                     D          1          1      0.545          1      0.995      0.995
                     F          2          2          1          1      0.995      0.995
                     G          5          5          1          1      0.995      0.971
                     H          3          3          1          1      0.995      0.951
                     I          2          2          1        0.5       0.75       0.75
                     J          4          4       0.93          1      0.995      0.911
                     K          4          4          1       0.75      0.875      0.875
                     

In [11]:
# Create comprehensive training visualizations
print(" Creating training performance visualizations...")

# Look for training results
results_dir = dirs['models'] / 'asl_yolov8_training'
results_csv = results_dir / 'results.csv'

if results_csv.exists():
    print(" Loading training history...")

    # Load training results
    df = pd.read_csv(results_csv)

    # Create comprehensive training dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Training & Validation Loss',
            'mAP Scores Over Time',
            'Precision & Recall',
            'Learning Rate Schedule',
            'Loss Components',
            'Accuracy Progression'
        ]
    )

    epochs = df.index + 1

    # 1. Training & Validation Loss
    if 'train/box_loss' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['train/box_loss'], name='Train Box Loss', line=dict(color='red')),
            row=1, col=1
        )
    if 'val/box_loss' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['val/box_loss'], name='Val Box Loss', line=dict(color='blue')),
            row=1, col=1
        )

    # 2. mAP Scores
    if 'metrics/mAP50(B)' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['metrics/mAP50(B)'], name='mAP@0.5', line=dict(color='green')),
            row=1, col=2
        )
    if 'metrics/mAP50-95(B)' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['metrics/mAP50-95(B)'], name='mAP@0.5-0.95', line=dict(color='orange')),
            row=1, col=2
        )

    # 3. Precision & Recall
    if 'metrics/precision(B)' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['metrics/precision(B)'], name='Precision', line=dict(color='purple')),
            row=2, col=1
        )
    if 'metrics/recall(B)' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['metrics/recall(B)'], name='Recall', line=dict(color='brown')),
            row=2, col=1
        )

    # 4. Learning Rate
    if 'lr/pg0' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['lr/pg0'], name='Learning Rate', line=dict(color='black')),
            row=2, col=2
        )

    # 5. Loss Components
    loss_cols = ['train/box_loss', 'train/cls_loss', 'train/dfl_loss']
    colors = ['red', 'green', 'blue']

    for i, (col, color) in enumerate(zip(loss_cols, colors)):
        if col in df.columns:
            fig.add_trace(
                go.Scatter(x=epochs, y=df[col], name=col.split('/')[-1], line=dict(color=color)),
                row=3, col=1
            )

    # 6. Accuracy Progression (mAP50 with target line)
    if 'metrics/mAP50(B)' in df.columns:
        fig.add_trace(
            go.Scatter(x=epochs, y=df['metrics/mAP50(B)'], name='Accuracy (mAP@0.5)',
                      line=dict(color='green', width=3)),
            row=3, col=2
        )
        # Add target line at 90%
        fig.add_trace(
            go.Scatter(x=[1, len(epochs)], y=[0.9, 0.9], name='90% Target',
                      line=dict(color='red', dash='dash')),
            row=3, col=2
        )

    # Update layout
    fig.update_layout(
        title_text="YOLOv8 ASL Training Performance Dashboard",
        showlegend=True,
        height=1200,
        hovermode='x unified'
    )

    # Update x-axis labels
    for i in range(1, 4):
        for j in range(1, 3):
            fig.update_xaxes(title_text="Epoch", row=i, col=j)

    fig.show()

    # Print training summary
    print("\n Training Summary:")
    print(f"   Total Epochs: {len(df)}")

    if 'metrics/mAP50(B)' in df.columns:
        best_map = df['metrics/mAP50(B)'].max()
        best_epoch = df['metrics/mAP50(B)'].idxmax() + 1
        final_map = df['metrics/mAP50(B)'].iloc[-1]

        print(f"   Best mAP@0.5: {best_map:.3f} ({best_map*100:.1f}%) at epoch {best_epoch}")
        print(f"   Final mAP@0.5: {final_map:.3f} ({final_map*100:.1f}%)")

        if best_map >= 0.90:
            print("    TARGET ACHIEVED: Model reached 90%+ accuracy!")
        else:
            improvement_needed = (0.90 - best_map) * 100
            print(f"    Need {improvement_needed:.1f}% more accuracy to reach 90% target")

    # Save visualization
    viz_path = dirs['visualizations'] / 'training_dashboard.html'
    fig.write_html(str(viz_path))
    print(f"\n Training dashboard saved: {viz_path}")

else:
    print(" Training results not found. Skipping training visualizations.")
    print("   This might happen if training was interrupted or failed.")

 Creating training performance visualizations...
 Loading training history...



 Training Summary:
   Total Epochs: 100
   Best mAP@0.5: 0.968 (96.8%) at epoch 80
   Final mAP@0.5: 0.964 (96.4%)
    TARGET ACHIEVED: Model reached 90%+ accuracy!

 Training dashboard saved: /content/asl_project/visualizations/training_dashboard.html


In [12]:
# Create confusion matrix and per-class analysis
print(" Creating detailed performance analysis...")

# Test on validation set to get predictions
val_images_dir = unified_dir / 'val' / 'images'

if val_images_dir.exists():
    print("\n Running predictions on validation set...")

    # Get validation images
    val_images = list(val_images_dir.glob('*'))

    if val_images:
        # Sample subset for detailed analysis (to avoid memory issues)
        sample_size = min(500, len(val_images))
        sample_images = val_images[:sample_size]

        print(f"   Analyzing {len(sample_images)} validation images...")

        # Run predictions
        predictions = best_model.predict(
            source=[str(img) for img in sample_images],
            conf=0.25,
            iou=0.6,
            verbose=False
        )

        # Collect predictions and ground truth
        y_true = []
        y_pred = []
        class_accuracies = defaultdict(list)

        for i, (img_path, pred) in enumerate(zip(sample_images, predictions)):
            # Get ground truth from label file
            label_path = unified_dir / 'val' / 'labels' / f"{img_path.stem}.txt"

            if label_path.exists():
                with open(label_path, 'r') as f:
                    lines = f.readlines()

                # Get ground truth classes
                gt_classes = [int(line.strip().split()[0]) for line in lines if line.strip()]

                # Get predicted classes
                if pred.boxes is not None and len(pred.boxes) > 0:
                    pred_classes = pred.boxes.cls.cpu().numpy().astype(int)
                    confidences = pred.boxes.conf.cpu().numpy()

                    # For each ground truth, find best matching prediction
                    for gt_class in gt_classes:
                        y_true.append(gt_class)

                        # Find if this class was predicted
                        if gt_class in pred_classes:
                            # Get confidence for this class
                            class_mask = pred_classes == gt_class
                            max_conf = confidences[class_mask].max() if np.any(class_mask) else 0
                            y_pred.append(gt_class)
                            class_accuracies[gt_class].append(1)
                        else:
                            # Class not detected correctly
                            if len(pred_classes) > 0:
                                # Use highest confidence prediction
                                best_pred_idx = np.argmax(confidences)
                                y_pred.append(pred_classes[best_pred_idx])
                            else:
                                # No prediction
                                y_pred.append(-1)  # Use -1 for no detection
                            class_accuracies[gt_class].append(0)
                else:
                    # No detections for this image
                    for gt_class in gt_classes:
                        y_true.append(gt_class)
                        y_pred.append(-1)
                        class_accuracies[gt_class].append(0)

        if y_true and y_pred:
            print(f"   Processed {len(y_true)} annotations")

            # Create confusion matrix
            print("\n Generating confusion matrix...")

            # Get unique classes that appear in predictions
            unique_classes = sorted(list(set(y_true + y_pred)))
            class_names = []

            for cls_id in unique_classes:
                if cls_id == -1:
                    class_names.append('No Detection')
                elif 0 <= cls_id < len(yaml_config['names']):
                    class_names.append(yaml_config['names'][cls_id])
                else:
                    class_names.append(f'Class_{cls_id}')

            # Compute confusion matrix
            cm = confusion_matrix(y_true, y_pred, labels=unique_classes)

            # Create interactive confusion matrix
            fig_cm = go.Figure(data=go.Heatmap(
                z=cm,
                x=class_names,
                y=class_names,
                colorscale='Blues',
                showscale=True,
                text=cm,
                texttemplate="%{text}",
                textfont={"size": 10},
                hoverongaps=False
            ))

            fig_cm.update_layout(
                title='ASL Detection Confusion Matrix',
                xaxis_title='Predicted Class',
                yaxis_title='True Class',
                width=800,
                height=800
            )

            fig_cm.show()

            # Per-class accuracy analysis
            print("\n Per-class accuracy analysis:")

            class_stats = []

            for class_id in sorted(class_accuracies.keys()):
                if 0 <= class_id < len(yaml_config['names']):
                    class_name = yaml_config['names'][class_id]
                    accuracies = class_accuracies[class_id]

                    if accuracies:
                        class_acc = np.mean(accuracies)
                        sample_count = len(accuracies)

                        class_stats.append({
                            'Class': class_name,
                            'ID': class_id,
                            'Accuracy': class_acc,
                            'Samples': sample_count
                        })

            if class_stats:
                # Create DataFrame and sort by accuracy
                df_stats = pd.DataFrame(class_stats).sort_values('Accuracy', ascending=False)

                # Display top and bottom performing classes
                print("\n Top 10 performing classes:")
                for _, row in df_stats.head(10).iterrows():
                    print(f"   {row['Class']}: {row['Accuracy']:.2%} ({row['Samples']} samples)")

                print("\n Bottom 10 performing classes:")
                for _, row in df_stats.tail(10).iterrows():
                    print(f"   {row['Class']}: {row['Accuracy']:.2%} ({row['Samples']} samples)")

                # Create per-class accuracy chart
                fig_acc = px.bar(
                    df_stats,
                    x='Class',
                    y='Accuracy',
                    title='Per-Class Detection Accuracy',
                    color='Accuracy',
                    color_continuous_scale='RdYlGn',
                    hover_data=['Samples']
                )

                fig_acc.update_layout(
                    xaxis_tickangle=-45,
                    height=600,
                    yaxis_title="Accuracy (%)"
                )

                fig_acc.show()

                # Save detailed results
                results_path = dirs['results'] / 'class_performance.csv'
                df_stats.to_csv(results_path, index=False)
                print(f"\n Detailed results saved: {results_path}")

                # Overall statistics
                overall_acc = df_stats['Accuracy'].mean()
                print(f"\n Overall Statistics:")
                print(f"   Average class accuracy: {overall_acc:.2%}")
                print(f"   Best class: {df_stats.iloc[0]['Class']} ({df_stats.iloc[0]['Accuracy']:.2%})")
                print(f"   Worst class: {df_stats.iloc[-1]['Class']} ({df_stats.iloc[-1]['Accuracy']:.2%})")
                print(f"   Classes above 90%: {len(df_stats[df_stats['Accuracy'] >= 0.9])}")
                print(f"   Classes below 50%: {len(df_stats[df_stats['Accuracy'] < 0.5])}")

        else:
            print(" No valid predictions found for analysis")

    else:
        print(" No validation images found")

else:
    print(" Validation directory not found. Skipping detailed analysis.")

print("\n Performance analysis completed!")

 Creating detailed performance analysis...

 Running predictions on validation set...
   Analyzing 144 validation images...
   Processed 144 annotations

 Generating confusion matrix...



 Per-class accuracy analysis:

 Top 10 performing classes:
   B: 100.00% (9 samples)
   C: 100.00% (3 samples)
   G: 100.00% (5 samples)
   D: 100.00% (6 samples)
   E: 100.00% (4 samples)
   F: 100.00% (8 samples)
   J: 100.00% (8 samples)
   H: 100.00% (9 samples)
   L: 100.00% (4 samples)
   K: 100.00% (6 samples)

 Bottom 10 performing classes:
   U: 100.00% (7 samples)
   S: 100.00% (4 samples)
   Y: 87.50% (8 samples)
   P: 85.71% (7 samples)
   O: 85.71% (7 samples)
   A: 80.00% (5 samples)
   V: 80.00% (5 samples)
   M: 75.00% (8 samples)
   T: 66.67% (6 samples)
   I: 50.00% (2 samples)



 Detailed results saved: /content/asl_project/results/class_performance.csv

 Overall Statistics:
   Average class accuracy: 92.72%
   Best class: B (100.00%)
   Worst class: I (50.00%)
   Classes above 90%: 18
   Classes below 50%: 0

 Performance analysis completed!


In [1]:
# Create interactive image upload interface
print(" Image Upload & Prediction Interface")
print("Upload ASL images to test the trained model!\n")

# Function to process uploaded images
def process_uploaded_image(image_path, show_results=True):
    """Process uploaded image and return predictions with visualization"""

    try:
        # Run prediction
        results = best_model.predict(
            source=image_path,
            conf=0.25,  # Confidence threshold
            iou=0.6,    # IoU threshold
            verbose=False,
            save=False
        )

        # Load original image
        image = cv2.imread(str(image_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get predictions
        predictions = []

        if results and len(results) > 0:
            result = results[0]

            if result.boxes is not None and len(result.boxes) > 0:
                boxes = result.boxes.xyxy.cpu().numpy()
                confidences = result.boxes.conf.cpu().numpy()
                class_ids = result.boxes.cls.cpu().numpy().astype(int)

                # Process each detection
                for i in range(len(boxes)):
                    class_id = class_ids[i]
                    confidence = confidences[i]

                    if 0 <= class_id < len(yaml_config['names']):
                        class_name = yaml_config['names'][class_id]

                        predictions.append({
                            'class': class_name,
                            'confidence': confidence,
                            'box': boxes[i]
                        })

        if show_results:
            # Create visualization
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))

            # Original image
            axes[0].imshow(image_rgb)
            axes[0].set_title('Original Image')
            axes[0].axis('off')

            # Annotated image
            annotated_image = image_rgb.copy()

            for pred in predictions:
                box = pred['box']
                x1, y1, x2, y2 = box.astype(int)

                # Draw bounding box
                cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Add label
                label = f"{pred['class']}: {pred['confidence']:.2f}"
                cv2.putText(annotated_image, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            axes[1].imshow(annotated_image)
            axes[1].set_title('Predictions')
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

            # Print predictions
            if predictions:
                print("\n Detected ASL Signs:")
                for i, pred in enumerate(predictions, 1):
                    print(f"   {i}. {pred['class']} (confidence: {pred['confidence']:.2%})")

                # Implement capitalization logic
                detected_letters = [pred['class'] for pred in predictions if pred['class'].isalpha()]
                if detected_letters:
                    print("\n Capitalization Options:")
                    letters_text = ''.join(detected_letters)
                    print(f"   Lowercase: {letters_text.lower()}")
                    print(f"   Uppercase: {letters_text.upper()}")
                    print(f"   Title Case: {letters_text.title()}")
            else:
                print("\n No ASL signs detected. Try:")
                print("   - Better lighting")
                print("   - Clearer hand position")
                print("   - Different camera angle")

        return predictions

    except Exception as e:
        print(f" Error processing image: {e}")
        return []

# Function to handle file upload
def upload_and_predict():
    """Handle file upload and prediction"""
    print(" Please upload an ASL image (JPG, PNG formats supported)")

    try:
        uploaded = files.upload()

        for filename in uploaded.keys():
            print(f"\n Processing: {filename}")

            # Save uploaded file
            upload_path = dirs['predictions'] / filename
            with open(upload_path, 'wb') as f:
                f.write(uploaded[filename])

            # Process the image
            predictions = process_uploaded_image(upload_path)

            if predictions:
                print(f"\n Successfully processed {filename}")
            else:
                print(f"\n No detections in {filename}")

    except Exception as e:
        print(f" Upload error: {e}")

print(" Image upload interface ready!")
print("Run upload_and_predict() to start uploading images")
print("\nTips for best results:")
print("    Use clear, well-lit images")
print("    Ensure hand is clearly visible")
print("    Center the ASL sign in the image")
print("    Use images similar to training data")

 Image Upload & Prediction Interface
Upload ASL images to test the trained model!

 Image upload interface ready!
Run upload_and_predict() to start uploading images

Tips for best results:
    Use clear, well-lit images
    Ensure hand is clearly visible
    Center the ASL sign in the image
    Use images similar to training data


In [ ]:
# Interactive upload button
upload_button = widgets.Button(
    description=' Upload ASL Image',
    disabled=False,
    button_style='success',
    tooltip='Click to upload an ASL image for prediction',
    icon='upload'
)

output_widget = widgets.Output()

def on_upload_click(b):
    with output_widget:
        clear_output(wait=True)
        upload_and_predict()

upload_button.on_click(on_upload_click)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h3> ASL Image Prediction Interface</h3>"),
    widgets.HTML("<p>Upload your ASL images to test the trained model!</p>"),
    upload_button,
    output_widget
]))

print("\n Use the upload button above to test your ASL images!")

In [15]:
# Real-time webcam detection (Note: Limited in Colab environment)
print(" Real-Time ASL Detection Setup")
print("\n Note: Webcam access is limited in Google Colab.")
print("For full real-time detection, download this notebook and run locally.\n")

# Function for webcam detection (works locally)
def real_time_asl_detection(duration=30):
    """
    Real-time ASL detection using webcam
    This function works when running locally, not in Colab
    """
    print(f" Starting real-time ASL detection for {duration} seconds...")
    print("Press 'q' to quit early\n")

    try:
        import cv2
        from collections import deque, Counter
        import time

        # Initialize camera
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print(" Cannot access webcam. Make sure it's connected and not in use.")
            return

        # Detection history for stability
        detection_history = deque(maxlen=10)
        stable_detections = []

        start_time = time.time()
        frame_count = 0

        print(" Webcam initialized. Starting detection...")

        while time.time() - start_time < duration:
            ret, frame = cap.read()
            if not ret:
                print(" Failed to read from webcam")
                break

            frame_count += 1

            # Run detection every 3 frames for performance
            if frame_count % 3 == 0:
                # Run YOLOv8 prediction
                results = best_model.predict(
                    source=frame,
                    conf=0.3,
                    verbose=False,
                    save=False
                )

                current_detections = []

                if results and len(results) > 0:
                    result = results[0]

                    if result.boxes is not None and len(result.boxes) > 0:
                        boxes = result.boxes.xyxy.cpu().numpy()
                        confidences = result.boxes.conf.cpu().numpy()
                        class_ids = result.boxes.cls.cpu().numpy().astype(int)

                        # Draw detections on frame
                        for i in range(len(boxes)):
                            if confidences[i] > 0.3:  # Confidence threshold
                                class_id = class_ids[i]

                                if 0 <= class_id < len(yaml_config['names']):
                                    class_name = yaml_config['names'][class_id]
                                    confidence = confidences[i]

                                    # Bounding box coordinates
                                    x1, y1, x2, y2 = boxes[i].astype(int)

                                    # Draw bounding box
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                                    # Add label
                                    label = f"{class_name}: {confidence:.2f}"
                                    cv2.putText(frame, label, (x1, y1-10),
                                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                                    current_detections.append(class_name)

                # Update detection history
                detection_history.append(current_detections)

                # Check for stable detections (same sign detected multiple times)
                if len(detection_history) >= 5:
                    all_recent_detections = []
                    for det_list in list(detection_history)[-5:]:
                        all_recent_detections.extend(det_list)

                    if all_recent_detections:
                        most_common = Counter(all_recent_detections).most_common(1)[0]
                        if most_common[1] >= 3:  # Detected in at least 3 of last 5 frames
                            stable_sign = most_common[0]
                            if not stable_detections or stable_detections[-1] != stable_sign:
                                stable_detections.append(stable_sign)
                                print(f" Detected: {stable_sign}")

            # Add info overlay
            remaining = int(duration - (time.time() - start_time))
            cv2.putText(frame, f"Time: {remaining}s", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if stable_detections:
                detected_text = ' '.join(stable_detections[-10:])  # Show last 10
                cv2.putText(frame, f"Detected: {detected_text}", (10, 70),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            # Show frame
            cv2.imshow('ASL Real-Time Detection', frame)

            # Exit on 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n Detection stopped by user")
                break

        # Cleanup
        cap.release()
        cv2.destroyAllWindows()

        # Summary
        print(f"\n Session Summary:")
        print(f"   Duration: {time.time() - start_time:.1f} seconds")
        print(f"   Frames processed: {frame_count}")
        print(f"   Stable detections: {len(stable_detections)}")

        if stable_detections:
            print(f"\n Detected sequence: {' '.join(stable_detections)}")

            # Capitalization options
            letters = [s for s in stable_detections if s.isalpha()]
            if letters:
                text = ''.join(letters)
                print(f"\n  Text options:")
                print(f"   Lowercase: {text.lower()}")
                print(f"   Uppercase: {text.upper()}")
                print(f"   Title case: {text.title()}")
        else:
            print("\n No stable detections recorded")

    except ImportError:
        print(" OpenCV not available for webcam access")
    except Exception as e:
        print(f" Webcam detection error: {e}")

# Colab-specific message
print(" For Local Use:")
print("   1. Download this notebook")
print("   2. Run: real_time_asl_detection(30)")
print("   3. Point webcam at ASL signs")
print("   4. Press 'q' to quit")

print("\n For Colab Alternative:")
print("   Use the image upload interface above to test individual images")
print("   Take photos with your phone/webcam and upload them")

# Create a simple test with sample detection
def simulate_detection_demo():
    """Simulate real-time detection for demo purposes"""
    print("\n Simulating real-time detection...")

    # Simulate detection sequence
    simulated_sequence = ['H', 'E', 'L', 'L', 'O']

    for i, letter in enumerate(simulated_sequence):
        print(f"Frame {i+1}: Detected '{letter}' (confidence: 0.{85+i}%)")
        time.sleep(0.5)

    print("\n Simulated detection complete!")
    print(f"Detected word: {''.join(simulated_sequence)}")
    print(f"Capitalization options:")
    text = ''.join(simulated_sequence)
    print(f"   Lowercase: {text.lower()}")
    print(f"   Uppercase: {text.upper()}")
    print(f"   Title case: {text.title()}")

print("\n Try the simulation: simulate_detection_demo()")

 Real-Time ASL Detection Setup

 Note: Webcam access is limited in Google Colab.
For full real-time detection, download this notebook and run locally.

 For Local Use:
   1. Download this notebook
   2. Run: real_time_asl_detection(30)
   3. Point webcam at ASL signs
   4. Press 'q' to quit

 For Colab Alternative:
   Use the image upload interface above to test individual images
   Take photos with your phone/webcam and upload them

 Try the simulation: simulate_detection_demo()


In [16]:
import time
# Run the simulation demo
simulate_detection_demo()


 Simulating real-time detection...
Frame 1: Detected 'H' (confidence: 0.85%)
Frame 2: Detected 'E' (confidence: 0.86%)
Frame 3: Detected 'L' (confidence: 0.87%)
Frame 4: Detected 'L' (confidence: 0.88%)
Frame 5: Detected 'O' (confidence: 0.89%)

 Simulated detection complete!
Detected word: HELLO
Capitalization options:
   Lowercase: hello
   Uppercase: HELLO
   Title case: Hello
